<a href="https://colab.research.google.com/github/kachytronico/colab-PIA/blob/main/PIA_02_Tarea_Procesamiento_de_la_informacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIA Unidad 2: Preprocesamiento de la información

# Carga de datos y Librerías

## Importamos las librerias necesarias

In [6]:
import pandas as pd # manipulación y análisis de datos
import numpy as np # operaciones numéricas (matrices y arrays)
import matplotlib.pyplot as plt # visualización base de gráficos
import seaborn as sns # visualizaciones estadísticas y acceder a datasets

## Ejemplo Cargar Dataset desde Drive del FPD:
He provado montado el drive y para  poder leer el csv con `pd.read_csv()` **He vuscado el archivo y he copidado la ruta**


```
#Carga de datos desde Drive
from google.colab import drive
drive.mount('/content/drive')

# Carga del dataset una vez montado el Drive
df = pd.read_csv("/content/drive/Shareddrives/alf.ledesma con FPD/01_IA/01_PIA Programación de Inteligencia Artificial/PIA_UD_02/dataset.csv") # leemos el csv
df
```



## Traer el Dataset desde Github:

 El comando `wget` descarga el archivo `dataset.csv `directamente desde la URL proporcionada en GitHub.



In [4]:
!wget https://raw.githubusercontent.com/kachytronico/colab-PIA/refs/heads/main/PIA_Tarea_02/dataset.csv

--2025-11-26 11:36:01--  https://raw.githubusercontent.com/kachytronico/colab-PIA/refs/heads/main/PIA_Tarea_02/dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 533333 (521K) [text/plain]
Saving to: ‘dataset.csv.1’

dataset.csv.1       100%[===================>] 520.83K  --.-KB/s    in 0.04s   

2025-11-26 11:36:01 (14.1 MB/s) - ‘dataset.csv.1’ saved [533333/533333]



In [8]:
# Carga del dataset
df = pd.read_csv("dataset.csv") # leemos el csv
df.head()

,Unnamed: 0,index,age,gender,education,job,experience,salary,country,race
0,0,0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0,UK,White
1,1,1,28.0,Female,Master's,Data Analyst,3.0,65000.0,USA,Hispanic
2,2,2,NaN,Male,PhD,Senior Manager,15.0,150000.0,Canada,White
3,3,3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0,USA,Hispanic
4,4,4,52.0,Male,Master's,Director,20.0,200000.0,USA,Asian


# 1. AED (Punto 1)










In [9]:
df.describe()

,Unnamed: 0,index,age,experience,salary
count,6704.000000,6704.000000,6142.000000,6701.000000,6699.000000
mean,3351.500000,3351.500000,33.616412,8.094687,115326.964771
std,1935.422435,1935.422435,7.689423,6.059003,52786.183911
min,0.000000,0.000000,-1.000000,0.000000,350.000000
25%,1675.750000,1675.750000,28.000000,3.000000,70000.000000
50%,3351.500000,3351.500000,32.000000,7.000000,115000.000000
75%,5027.250000,5027.250000,38.000000,12.000000,160000.000000
max,6703.000000,6703.000000,62.000000,34.000000,250000.000000


In [10]:
df.columns

Index(['Unnamed: 0', 'index', 'age', 'gender', 'education', 'job',
       'experience', 'salary', 'country', 'race'],
      dtype='object')

## 1.1. Análisis Exploratorio de Datos (AED) Inicial con `.describe()`

Uso los estadísticos descriptivos de `df.describe()` para hacer un diagnóstico inicial de las variables numéricas. Esto me permite detectar la dispersión, los valores extremos (máximos) y confirmar la existencia de nulos antes de empezar el preprocesamiento.


Al analizar los resultados de `.describe()`, me concentro en la media, el conteo y la dispersión.

### 1. Diagnóstico de la Columna Objetivo (`Salario`)

*   **Es Regresión:** El `Salario` es un valor real (con decimales o muy variable), lo que confirma que el problema es de **regresión** .
*   **Valores Atípicos:** Miro la diferencia entre el **Percentil 75%** y el **valor Máximo**. Si el valor máximo es muchísimo más grande que el P75, eso me indica que hay **valores atípicos** (*outliers*) extremos que deberé tratar en el Punto 2.

### 2. Detección de Nulos

*   **Conteo (`count`):** Compruebo el `count` de todas las columnas numéricas.
*   **Plan de Nulos:** Si el `count` es menor al número total de filas, tengo valores nulos. Esto me obliga a planificar la imputación (relleno) o la eliminación de esos nulos en el Punto 3, ya que los modelos de IA no trabajan con datos faltantes .

### 3. Dispersión y Desviación Típica (`std`)

*   **Dispersión:** Si la desviación típica (`std`) es demasiado alta (por ejemplo, más de 1/3 de la media), indica que los datos están muy dispersos, y debo tener cuidado en cómo los escalo.


## 1.2. Tipos de Datos y Nulos: `df.info()`
Utilizo `.info()` para obtener el resumen conciso del *DataFrame* (total de entradas y tipos de datos por columna). Esto es crucial para planificar dos acciones de preprocesamiento: el tratamiento de nulos (Punto 3) y la codificación de variables de texto (Punto 5).


In [11]:
# Muestro el resumen de las columnas, el tipo de dato y el conteo de valores no nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 0 to 6703
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  6704 non-null   int64  
 1   index       6704 non-null   int64  
 2   age         6142 non-null   float64
 3   gender      6702 non-null   object 
 4   education   6701 non-null   object 
 5   job         6702 non-null   object 
 6   experience  6701 non-null   float64
 7   salary      6699 non-null   float64
 8   country     6704 non-null   object 
 9   race        6704 non-null   object 
dtypes: float64(3), int64(2), object(5)
memory usage: 523.9+ KB


### Análisis de `df.info()`

Este resumen me obliga a tomar decisiones inmediatas para la limpieza.

### 1. Planificación del Tratamiento de Nulos

**La mayoría de los modelos de IA dan error con valores nulos, por lo que debo eliminarlos o rellenarlos .**

*   **Columna con Nulos Críticos (Ej. ``):** Si identifico una columna con un alto porcentaje de nulos (más del 50%), mi justificación será proponer su **eliminación inmediata** en el Punto 2, ya que no aporta suficiente información y la imputación no sería válida .
*   **Columnas con Nulos Manejables (Ej.):** Si la cantidad de nulos es menor al 50% (por ejemplo, la variable `Age` tiene xxx nulos de xxx filas en el ejemplo del profesor, lo cual es significativo pero manejable), mi estrategia en el Punto 3 será la **imputación** (rellenarlos con la media, mediana o moda) .
*   **Nulos muy pequeños:** Si son muy pocos (menos del 5%), la solución más sencilla es borrarlos con `dropna()` .

### 2. Planificación de la Codificación

**Los modelos de IA solo trabajan con números .**

*   **Variables Categóricas (`object`):** Las columnas que aparecen como tipo `object` contienen texto (ej. ) . Debo transformarlas a números mediante **codificación** (Punto 5).
*   **Variables Numéricas (`int64` y `float64`):** Estas son aptas para el modelado, aunque algunas pueden requerir **escalamiento** (normalización o estandarización) en el Punto 6 para evitar que los números grandes dominen el entrenamiento.


In [ ]:
df[""]